In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import xarray as xr

In [2]:
byrd_bedmap_points = pd.read_csv('/home/kim/ice_thickness/data/byrd_200km.csv')
path_to_file_server = 'data/byrd_bedmachine.nc'
byrd_bedmachine = xr.open_dataset(path_to_file_server)

In [54]:
byrd_bedmachine.bed.values.shape

(400, 400)

# Count per grid cell

Assign IDs and groupby pixel IDs to then compute std, count and mean
Sample minimum one per grid cell

In [48]:
byrd_bedmap_points_assigned = pd.read_csv('/home/kim/ice_thickness/data/byrd_200km_assigned_to_pixels.csv')

## Create IDs

In [49]:
# Make categocial column based on grid cell index
byrd_bedmap_points_assigned["id_500"] = byrd_bedmap_points_assigned["x_pixel_center"].astype(int).astype(str) + "_" + byrd_bedmap_points_assigned["y_pixel_center"].astype(int).astype(str)

# Group by

400 x 400 cells

In [59]:
# Rate of grid cells with assigned points
byrd_bedmap_points_assigned.groupby("id_500").size().sort_values(ascending = False).shape[0]/(400*400)

0.17030625

In [69]:
means_per_pixel = byrd_bedmap_points_assigned.loc[:, ["id_500", "t"]].groupby("id_500").mean()

std_per_pixel = byrd_bedmap_points_assigned.loc[:, ["id_500", "t"]].groupby("id_500").std()
# std is NaN for grid cells with only one point assigned

sem_per_pixel = byrd_bedmap_points_assigned.loc[:, ["id_500", "t"]].groupby("id_500").sem()
# std / np.art(count)

count_per_pixel = byrd_bedmap_points_assigned.loc[:, ["id_500", "t"]].groupby("id_500").count()

combined_metrics = pd.merge(pd.merge(means_per_pixel, std_per_pixel, on = "id_500", suffixes = ("_mean", "_std")), count_per_pixel, on = "id_500")
# Afterthought: order should be identicla so don't need to merge

# Add this to a xarray dataset